Resultados esperados:
 - input: 
    - avg_runtime
    - input_complexity
 - output: 
    - p_cores


In [1]:
import os
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame()
pkgs = os.listdir("./dataset")

for pkg in pkgs:
    path = "dataset/{}/{}-pthreads_g.json".format(pkg, pkg)
    df_tmp = pd.read_json(path)
    df = pd.concat([df, df_tmp])

In [3]:
df = pd.read_json("dataset/freqmine/freqmine-pthreads_g.json")

In [6]:
def calculate_median(data_list, start_index, end_index):
    return np.median([data['stop_time'] - data['start_time'] for data in data_list[start_index:end_index]])

In [7]:
def extract_avg_runtime(df):
    data_list = df[df['data'].notna()]['data']
    keys = data_list.index

    avgs = []
    BATCH_SIZE = 10

    for i in range(0, len(data_list), BATCH_SIZE):
        avg = calculate_median(data_list, i, i + BATCH_SIZE)
        config = keys[i].split(";")
        avgs.append((config[0], config[1], avg))

    df_data = pd.DataFrame(avgs, columns=['cores', 'input', 'average_runtime'])
    return df_data

In [8]:
def extract_avg_runtime_without_agg(df):
    data_list = df[df['data'].notna()]['data']
    keys = data_list.index

    avgs = []
    BATCH_SIZE = 1

    for i in range(0, len(data_list), BATCH_SIZE):
        avg = calculate_median(data_list, i, i + BATCH_SIZE)
        config = keys[i].split(";")
        avgs.append((config[0], config[1], avg))

    df_data = pd.DataFrame(avgs, columns=['cores', 'input', 'average_runtime'])
    return df_data

In [9]:
df_data = extract_avg_runtime(df)

In [10]:
df_data['input'] = df_data['input'].astype(int)
df_data['cores'] = df_data['cores'].astype(int)

In [11]:
# Calculando Speedup e Eficiência
inputs = len(df_data['input'].unique())

df_clean = pd.DataFrame()

for i in range(inputs):
    df_input = df_data[df_data['input'] == i]
    reference_speedup = df_input[df_input['cores'] == 1]['average_runtime'].values[0]
    df_input['speedup'] = reference_speedup / df_input['average_runtime']
    df_input['efficiency'] = df_input['speedup'] / df_input['cores']
    df_clean = pd.concat([df_clean, df_input], ignore_index=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_18880\3757201144.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['speedup'] = reference_speedup / df_input['average_runtime']
C:\Users\PC\AppData\Local\Temp\ipykernel_18880\3757201144.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input['efficiency'] = df_input['speedup'] / df_input['cores']
C:\Users\PC\AppData\Local\Temp\ipykernel_18880\3757201144.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


In [11]:
df_clean['mem_ram'] = 128 # GB
df_clean['freq_mem'] = 2133 # MHz
df_clean['freq_cpu'] = 2.3 # GHz
df_clean['mem_l3'] = 40 # MB

In [12]:
df_clean.to_csv('dataset/clean_data.csv', index=False)